<a href="https://colab.research.google.com/github/fjaviergallucci/mia-ao-ag3/blob/master/Francisco_Gallucci_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Francisco Gallucci  <br>
Url: <br>
Problema:
>3. Combinar cifras y operaciones

Descripción del problema:(copiar enunciado)
-   El problema consiste en analizar el siguiente problema y diseñar un algoritmo que lo resuelva.
-   Disponemos de las 9 cifras del 1 al 9 (excluimos el cero) y de los 4 signos básicos de las
operaciones fundamentales: suma(+), resta(-), multiplicación(*) y división(/)
-   Debemos combinarlos alternativamente sin repetir ninguno de ellos para obtener una cantidad
dada. Un ejemplo sería para obtener el 4:
-   Debe analizarse el problema para encontrar todos los valores enteros posibles planteando las siguientes cuestiones:
-   ¿Qué valor máximo y mínimo se pueden obtener según las condiciones del problema?
-   ¿Es posible encontrar todos los valores enteros posibles entre dicho mínimo y máximo ?
-   Nota: Es posible usar la función de python “eval” para evaluar una expresión:
....

(*) La respuesta es obligatoria

# Codigo

## Global functions and constants

In [1]:
import random
from itertools import permutations
import time
from itertools import chain
from itertools import zip_longest
import re
import copy

In [2]:
"""Region de funciones y constantes"""
_NUMBERS = '123456789'
_OPERATIONS = '+-*/'
_EXPRESSION_PATTERN = r'^[1-9]{1}[\-\+\*\/]{1}[1-9]{1}[\-\+\*\/]{1}[1-9]{1}[\-\+\*\/]{1}[1-9]{1}[\-\+\*\/]{1}[1-9]$'

def generate_number_yield():
    """Genera todas las posibles permutaciones de numeros

    Returns:
      Yield con el numero o permutacion generada.
    """
    for number in permutations(_NUMBERS, 5):
        yield number


def generate_operations_yield():
    """Esta funcion generara todas las posibles permutaciones de operadores

    Returns:
      Yield con la permutacion generada de los operadores
    """
    for operation in permutations(_OPERATIONS, 4):
        yield operation


def generate_expresion(number: str, operators: str):
    """Genera la expresion dado los numeros y los operadores

    La funcion recorre ambos strings y genera una combinacion de ambos respetando el orden y la estructura esperada

    Args:
      number: Permutacion de los numeros, por ejemplo "37586"
      operators: Permutacion de los operadores, por ejemplo: "+/*-"

    Returns:
      Expression generada
    """
    return "".join((filter(lambda x: x != '', chain.from_iterable(
        zip_longest(number, operators, fillvalue='')))))


def expression_is_valid(expression: str):
    """Valida una expresion dada que cumpla con las reglas

    Args:
      expression: Expresion a ser evaluada

    Returns:
      True si la expresion es correcta y cumple con las reglas
    """
    unique_genes = "".join(set(expression))
    length_valid = len(unique_genes) == 9
    pattern_valid = re.match(_EXPRESSION_PATTERN, expression)
    operators_valid = all(operator in unique_genes for operator in _OPERATIONS)
    numbers_valid = len([number for number in _NUMBERS if number in unique_genes]) == 5
    return length_valid and pattern_valid and operators_valid and numbers_valid

## Fuerza Bruta
Sub-problema 1: Dado un número, generar una expresion combinando numeros y operaciones<br>

Sub-problema 2: Generar todas las posibles combinaciones y:<br>
2.1: Buscar max<br>
2.2: Buscar min<br>
2.3: Valores enteros intermedios<br>

In [3]:
def find_expressions(target: int = None):
    """Funcion principal que genera todas las permutaciones o expresiones posibles y devuelve:

    Args:
      target: El numero para el cual se desan generar expresiones. Si no se pasa el target, se generaran todas las posibles permutaciones. Si se define el target, se devolveran solo las permutaciones que al ser evaluadas den ese numero

    Returns:
      number_permutations: Todas las permutaciones encontradas
      integer_permutations: Permutaciones cuya evaluaciones dan numeros enteros
      max: Permutacion que devuelve valor maximo
      min: Permutacion que devuelve valor minimo
    """
    number_permutations = {}
    integer_permutations = {}
    max = "0"
    min = "0"
    # Como las permutaciones de numeros son cientos de miles, usamos un generador para ahorrar memoria y tiempo de procesamiento
    for number in generate_number_yield():
        # En el caso particulas de las operaciones se puede pasar este generador a un lista. Pero no hace ninguna diferencia en recursos. Mejor seguir usando generadores
        for operation in generate_operations_yield():
            # Como ya tenemos todas las posibles permutaciones de operaciones. Lo que hay que hacer es jutnar el numero generado con el orden de los operadores y asi se obtienen todas las permutacioens de un numero con los operadores
            expression = generate_expresion(number, operation)
            value = eval(expression)

            # Determinamos si guardar o no la expresion. Las condiciones para que sea verdadero son:
            # Si no estoy buscando valor
            # Si estoy bsucando un target, entonces que sea igual al target
            if (target is not None and value == target) or target is None:
                number_permutations[expression] = value
                # Verificamos si es el nuevo maximo
                if value > eval(max):
                    max = expression
                # Verificamos si es el nuevo minimo
                if value < eval(min):
                    min = expression
                # Lo guardamos si el resultado de la expresion es entero
                if abs(value - int(value)) == 0.0:
                    integer_permutations[expression] = value
    # print(number_combinations)
    return number_permutations, integer_permutations, max, min


## Algoritmo Genetico
Sub-problema 1: Dado un número, generar una expresion combinando numeros y operaciones<br>

Sub-problema 2: Generar todas las posibles combinaciones y:<br>
2.1: Buscar max<br>
2.2: Buscar min<br>
2.3: Valores enteros intermedios<br>

#### Constantes y funciones generales

In [4]:
# Constantes
_GENERATIONS_HISTORY = {}
_POPULATION_SIZE = 100
_MAX_GENERATIONS = 100


def get_fitness(individual: str):
    """Calcular fitness

    Calcula el fitness de un individuo, el fitness en nuestro caso es simplemente la evaluacion de la expresion

    Args:
      individual: Expresion a evaluar

    Returns:
      El fitness del individuo
    """
    try:
        return eval(individual)
    except:
        return 0


def population_is_valid(expression: str):
    """Valida la poblacion

    Valida que los genes que poseen todos los individuos de la poblacion, tienen todos los numeros y operadores al menos una vez

    Args:
      expression: String que contiene todos los genes sin repetir de todos los individuos de la poblacion a evaluar

    Returns:
      Verdaro si la expresion contiene todos los numeros y operadores definidos
    """
    return all(operator in expression for operator in _OPERATIONS) and all(number in expression for number in _NUMBERS)


def order_population_by_fitness(population: dict, reverse_order: bool = True):
    """Ordena los sujetos de la poblacion en base al fitness

    Ordenar la poblacion por el fitness
    El fitness es el valor de la clave en el diccionario
    population.items() nos retorna tuplas por ejemplo ('1+2-3*4/5', 0.6), por eso ordenamos en el indice [1] que es fitness de esa expresion

    Args:
      population: Poblacion a ordenar
      reverse_order: Invierte el orden

    Returns:
        Poblacion ordenada por fitness
    """
    return dict(sorted(population.items(), key=lambda i: i[1], reverse=reverse_order))


def get_population_score(population: dict):
    """Calcula el score de la poblacion

    El score viene dado por la suma total de los valores absolutos de los fitness de cada individuo

    Args:
      population: Poblacion

    Returns:
        Suma de los fitness
    """
    return sum(map(abs, population.values()))


def get_individuals_with_integer_fitness(population: dict):
    """Devuelve los individuos cuyo fitness es entero

    Calcula el fitness de un individuo, el fitness en nuestro caso es simplemente la evaluacion de la expresion

    Args:
      individual: Expresion a evaluar

    Returns:
      El fitness del individuo
    """
    individuals = dict(filter(lambda i: abs(
        i[1] - int(i[1])) == 0.0, population.items()))
    return individuals


def find_individuals_by_fitness(population: dict, target_value: int):
    """Busca individuos por un fitness

    Busca todos los individuos de una poblacion que tengan el fitness buscado

    Args:
      population: Poblacion
      target_value: Fitness a buscar

    Returns:
        Diccionario con todos los individuos con el fitness buscado
    """
    return dict(filter(lambda i: i[1] == target_value, population.items()))



#### Poblacion inicial

In [5]:
def build_initial_population(population_size: int = 100, target_value: int = None):
    """Construye poblacion inicial

    Generamos una poblacion inicial aleatoria. Si el parametro target_value es pasado, los individuos generados que tengan ese fitness NO seran agregados a la poblacion, para garantizar que el AG generara al menos una generacion mas

    Args:
      population_size: Tamaño de la poblacion, por defecto 100
      target_value: Fitness a buscar

    Returns:
      Diccionario con los individuos y su fitness
    """
    population_genes = ""
    individuals = {}
    # Repetir mientras:
    # La poblacion aun no contenga todos los numeros y operadores
    # El tamaño de la poblacion sea menor al que queremos
    while(not population_is_valid(population_genes) or len(individuals) < population_size):
        random_numbers = random.sample(_NUMBERS, 5)
        # No usamos el generador en este caso porque vamos a tomar un numero aleatorio de permutaciones de todo el unvierso posible. Y en cada vuelta repetimos. Como son pocos, no hay impacto
        operations = list(permutations(_OPERATIONS, 4))
        random_operations_length = random.randrange(1, len(operations))
        random.shuffle(operations)
        for j in range(0, random_operations_length):
            # Generamos la expresion
            expression = generate_expresion(random_numbers, operations[j])
            fitness = get_fitness(expression)

            # Si se paso un target_value y el individuo tiene ese fitness, no se guarda. Leer doc arriba
            if (target_value is not None and target_value != fitness) or (target_value is None):
                # Guarda el individuo y su fitness en un diccionario
                individuals[expression] = fitness
                # Saca los genes del individuo y los agrega a los genes unicos de la poblacion para luego verificar que la poblacion tiene todos los numeros y operadores
                population_genes = "".join(set(population_genes + expression))
    return individuals


#### Seleccion

In [33]:
def select_individuals_by_ranking(population: dict):
    """Seleccion de individuos por fitness

    Seleccionamos en base al fitness del individuo (Ranking), recordando que el fitness de una expresion esta como valor dentro del diccionario. Seleccionamos 1/4 de los mayores y 1/4 de los menores

    Args:
      population: Poblacion

    Returns:
        Individuos seleccionados
    """
    population = order_population_by_fitness(population)
    selected_individuals = list(population.items())
    selected_individuals = dict(
        selected_individuals[:_POPULATION_SIZE//4] + selected_individuals[(_POPULATION_SIZE//4)*-1:])
    return selected_individuals


def select_gene_from_random_parents(parents: list, gene_index: int):
    """Selecciona los genes de los padres en una i-esima posicion

    Args:
      parents: Lista con los dos padres
      gene_index: Indice del gen que sacaremos de ambos padres

    Returns:
        Tupla con dos valores, el primero es el gen del padre A, y el segundo el gen del padre B
    """
    # Seleccionamos al azar el indice del que sera el padre a. 0-50 para padre a, 51-100 para padre b
    random_parent = 0 if random.randint(0, 100) < 51 else 1
    parent_a = parents[random_parent]
    parent_b = parents[abs(random_parent-1)]
    return parent_a[gene_index], parent_b[gene_index]


def select_random_parents(population: dict):
    """Selecciona padres al azar de una poblacion

    Args:
      population: Poblacion de la que se seleccionaran los padres

    Returns:
        Padres seleccionados
    """
    parent_a_expression, _ = random.choice(list(population.items()))
    parent_b_expression, _ = random.choice(list(population.items()))
    return parent_a_expression, parent_b_expression


#### Cruce

In [34]:
def uniform_crossover(parents: list):
    """Cruce uniforme

    Realizamos el cruce de los padres usando uniform crossover. Las expresiones tienen un conjunto de 9 caracteres, serian 9 genes. Como es aleatorio, cada cruce verificamos que la expresion sea una expresion correcta.
    Debido a las restricciones del problema. Vamos a repetir la seleccion aleatoria del padre, si el gen en la posicion i-esima de los padres, ya se encuentra en los hijos. Si reintentamos 20 veces y aun no se consigue un gen candidato, regresamos a los mismos padres

    Args:
      parents: Lista con los dos padres

    Returns:
        Nuevos hijos generados con cruce uniforme
    """
    while True:  # seleccion aleatoria del padre para el gen i
        individual_a = ""
        individual_b = ""
        for gene_index in range(0, 9):
            # Seleccion aleatoria del padre
            retries = 0
            gene_parent_a, gene_parent_b = select_gene_from_random_parents(
                parents, gene_index)
            # Repeticion si los hijos ya tienen el gen elegido
            while gene_parent_a in individual_a or gene_parent_b in individual_b:
                if retries > 20:
                    return parents[0], parents[1]
                gene_parent_a, gene_parent_b = select_gene_from_random_parents(
                    parents, gene_index)
                retries += 1

            # Agregamos el gen al hijo
            individual_a += gene_parent_a
            individual_b += gene_parent_b
        # Retornamos a los hijos solo si sus expresiones son validas
        if expression_is_valid(individual_a) and expression_is_valid(individual_b):
            return individual_a, individual_b
    return None, None


def one_point_crossover(parents: list):
    """Cruce uniforme

    Realizamos el cruce de los padres usando 1-point crossover. Las expresiones tienen un conjunto de 9 caracteres, serian 9 genes. El punto se elige aleatoriamente y se generan los dos hijos de los dos padres

    Args:
      parents: Lista con los dos padres

    Returns:
        Nuevos hijos generados con cruce uniforme
    """
    while True:  # seleccion aleatoria del padre para el gen i
        # Seleccionamos el punto en la secuencia del cromosoma
        random_point = random.randint(0, len(parents[0])-1)

        # Copiamos los genes a los hijos
        individual_a = parents[0][:random_point] + parents[1][random_point:]
        individual_b = parents[1][:random_point] + parents[0][random_point:]

        # Retornamos a los hijos solo si sus expresiones son validas
        if expression_is_valid(individual_a) and expression_is_valid(individual_b):
            return individual_a, individual_b
    return None, None


def generate_children_from_parents(parent_a: str, parent_b: str, target_generation: dict):
    """Genera hijos a partir de dos padres

    Se intenta generar dos hijos a partir de los dos padres. Sin embargo los hijos podrian ya existir en la nueva generacion. Por lo tanto se controla generacion infinita con una condicion, si la cantidad de ciclos supera la (cantidad de genes * 2) se rompe el ciclo y se retornan los mismos padres.

    Abstraemos del algoritmo principal la tecnica de cruce, si queremos cambiarla, la cambiamos aqui

    Args:
      parent_a: Padre A
      parent_a: Padre B
      target_generation: Generacion a la que perteneceran los hijos

    Returns:
        Hijos generados
    """
    children_a = ""
    children_b = ""
    children_exists = True
    number_of_attempts = 0
    while(children_exists):
        children_a, children_b = uniform_crossover([parent_a, parent_b])
        children_exists = children_a in target_generation.keys(
        ) or children_a in target_generation.keys()
        if number_of_attempts > len(parent_a)*2:
            return parent_a, parent_b
        number_of_attempts += 1
    return children_a, children_b

#### Mutacion

In [8]:
def mutate_population(population: dict):
    """Mutar un individuo de la poblacion

    Se selecciona un inviduo al azar y luego para la mutacion seleccionamos dos caracteres al azar de la expresion y los intercambiamos. Este proceso se repetira hasta que el individuo o expresion generada, sea correcta. Luego el indivudo antiguo es eliminado de a poblacion e insertado el nuevo o mutado

    Args:
      population: Poblacion

    Returns:
        Poblacion con un individuo mutado
    """
    individual, _ = random.choice(list(population.items()))
    del population[individual]
    while True:
        gene_1 = random.randint(0, len(individual)-1)
        gene_2 = random.randint(0, len(individual)-1)
        while gene_2 == gene_1:
            gene_2 = random.randint(0, len(individual)-1)

        # Hacemos el cruce intercambiando los dos caracteres de la expresion
        new_individual = individual[0:gene_1] + individual[gene_2] + \
            individual[gene_1+1:gene_2] + \
            individual[gene_1] + individual[gene_2+1:]

        if expression_is_valid(new_individual):
            population[new_individual] = get_fitness(new_individual)
            return population


#### Algoritmo genetico

In [41]:
def genetic_algorithm(initial_population: dict, target_value: int = None):
    """Algoritmo genetico

    Si target_value es definido, generara generaciones hasta conseguir un indivudo que tenga de fitness el valor buscado. Si target_value no es definido, generara generaciones hasta el maximo permitido

    Args:
      population: Poblacion inicial
      target_value: Fitness a buscar

    Returns:
        Poblacion final
    """
    global _GENERATIONS_HISTORY
    current_generation = 1
    population = copy.deepcopy(initial_population)

    while(current_generation < _MAX_GENERATIONS):
        # seleccionamos los mejores individuos por ranking
        selected_individuals = select_individuals_by_ranking(population)

        # Nueva generacion vacia
        new_generation = {}
        # Ciclo para seleccionar padres al azar y cruzarlos. Los hijos son agregados a la generacion nueva
        while(len(new_generation) < _POPULATION_SIZE):
            parent_a_expression, parent_b_expression = select_random_parents(selected_individuals)
            children_a, children_b = generate_children_from_parents(
                parent_a_expression, parent_b_expression, new_generation)
            # Agregar individuos a poblacion
            new_generation[children_a] = get_fitness(children_a)
            new_generation[children_b] = get_fitness(children_b)

        # Guardamos la poblacion actual con su score para historia
        _GENERATIONS_HISTORY[str(current_generation)] = {
            "population": list(population.keys()), "score": get_population_score(population)}

        print(f'Score poblacion {current_generation}: {_GENERATIONS_HISTORY[str(current_generation)]["score"]}')

        # Mutamos y hacemos la nueva generacion la poblacion actual
        population = mutate_population(new_generation)
        current_generation += 1

        # Si estamos buscando un individuo en particual, caso de expresion dado un numero
        # Retornamos una poblacion solo con el individuo encontrado
        if target_value is not None:
            result = find_individuals_by_fitness(population, target_value)
            if len(result.items()):
                return result, True

    return population, False

#### Funcion sub-problema 1

In [26]:
def find_individual_by_fitness_from_random_initial_population(target_value:int):
    """Ejecucion AG para conseguir inviduo con fitness especifico """
    global _GENERATIONS_HISTORY
    population = build_initial_population(_POPULATION_SIZE, target_value)
    start_time = time.time()
    population, found = genetic_algorithm(population, target_value)
    end_time = time.time()
    # Imprimir datos en caso de buscar un numero particular
    if found:
        print(
            f'*** Poblaciones generadas hasta encontrar individuo ***: {len(_GENERATIONS_HISTORY)}')
        print("Tiempo total: --- %s segundos ---" % (end_time - start_time))
        print(
            f'Individuos encontrados {len(population.items())}.\nIndividuos:')
        print(population.items())
    else:
        print(f'Individuo no enontrado')
    print()

#### Funcion sub-problema 2

In [28]:
def find_max_min_integers_from_random_population():
    """Ejecucion AG para conseguir expresiones aleatorias, max, min y enteros """
    global _GENERATIONS_HISTORY
    # Poblacion inicial
    population = build_initial_population(_POPULATION_SIZE)
    initial_population = order_population_by_fitness(population)
    start_time = time.time()
    population, _ = genetic_algorithm(population)
    end_time = time.time()

    # Imprimir datos de poblacion inicial para comparar con la final
    print()
    print("*** Poblacion inicial ***")
    print(
        f'Score poblacion inicial: {get_population_score(initial_population)}')
    max = list(initial_population)[0]
    min = list(initial_population)[-1]
    print(f'Max = {max} = {initial_population[max]}')
    print(f'Min = {min} = {initial_population[min]}')
    print()

    ###### Generar expresiones aleatorias para hallar max, min y enteros #######
    print()
    print(f'*** Poblaciones generadas ***: {len(_GENERATIONS_HISTORY)}')
    print("Tiempo total: --- %s segundos ---" % (end_time - start_time))
    print(f'Score poblacion final: {get_population_score(population)}')
    population = order_population_by_fitness(population)
    integer_individuals = get_individuals_with_integer_fitness(population)
    max = list(population)[0]
    min = list(population)[-1]
    print(f'Max = {max} = {population[max]}')
    print(f'Min = {min} = {population[min]}')
    print(
        f'cantidad idividuos con fitness entero {len(integer_individuals.items())}. Individuos: ')
    print(integer_individuals.items())
    print()


# Corridas

### Corrida sub-problema 1 con fuerza bruta

In [17]:
_NUMBER_TO_SEARCH = 55
start_time = time.time()
expressions, integer_expressions, max, min = find_expressions(_NUMBER_TO_SEARCH)
print("--- %s segundos ---" % (time.time() - start_time))
print(f'Cantidad de expresiones encontradas: {len(expressions)}')
print(f'Primeras 10 expresiones encontradas:')
count = 0
for expresion in expressions:
    if count <= 10:
        count += 1
        print(expresion)

--- 2.8994085788726807 segundos ---
Cantidad de expresiones encontradas: 392
Primeras 10 expresiones encontradas:
1-4/2+7*8
1-4/2+8*7
1-6/3+7*8
1-6/3+8*7
1+7*8-4/2
1+7*8-6/3
1+8*7-4/2
1+8*7-6/3
2/1-3+7*8
2/1-3+8*7
2/1+7*8-3


### Corrida sub-problema 2 con fuerza bruta

In [13]:
start_time = time.time()
expressions, integer_expressions, max, min = find_expressions()
print("--- %s segundos ---" % (time.time() - start_time))
print(f'Cantidad de expresiones: {len(expressions)}')
if max in expressions:
    print(f'Max = {max} = {expressions[max]}')
if min in expressions:
    print(f'Min = {min} = {expressions[min]}')
print(f'Cantidad de expresiones con valores enteros: {len(integer_expressions)}')
print(f'Primeras 10 expresiones con valores enteros encontradas:')
count = 0
for expresion in expressions:
    if count <= 10:
        count += 1
        print(expresion)

--- 6.376831531524658 segundos ---
Cantidad de expresiones: 362880
Max = 7-1/6+8*9 = 78.83333333333333
Min = 1+2/7-8*9 = -70.71428571428571
Cantidad de expresiones con valores enteros: 90000
Primeras 10 expresiones con valores enteros encontradas:
1+2-3*4/5
1+2-3/4*5
1+2*3-4/5
1+2*3/4-5
1+2/3-4*5
1+2/3*4-5
1-2+3*4/5
1-2+3/4*5
1-2*3+4/5
1-2*3/4+5
1-2/3+4*5


### Corrida sub-problema 1 con algoritmo genetico

In [46]:
_GENERATIONS_HISTORY = {}
_NUMBER_TO_SEARCH = 12
find_individual_by_fitness_from_random_initial_population(_NUMBER_TO_SEARCH)

Score poblacion 1: 1663.096031746032
Score poblacion 2: 2050.7583333333328
*** Poblaciones generadas hasta encontrar individuo ***: 2
Tiempo total: --- 0.007928133010864258 segundos ---
Individuos encontrados 1.
Individuos:
dict_items([('4+5*3-7/1', 12.0)])



### Corrida sub-problema2 con algoritmo genetico

In [45]:
_GENERATIONS_HISTORY = {}
find_max_min_integers_from_random_population()

Score poblacion 1: 1835.9726190476185
Score poblacion 2: 2640.500000000001
Score poblacion 3: 3483.9654761904753
Score poblacion 4: 3943.9182539682533
Score poblacion 5: 4448.2182539682535
Score poblacion 6: 5007.388888888889
Score poblacion 7: 5221.761904761905
Score poblacion 8: 5340.791666666667
Score poblacion 9: 5468.402777777779
Score poblacion 10: 5859.152777777777
Score poblacion 11: 6033.333333333333
Score poblacion 12: 6067.220238095239
Score poblacion 13: 6170.531746031747
Score poblacion 14: 6285.992063492063
Score poblacion 15: 6387.839285714284
Score poblacion 16: 6745.440476190478
Score poblacion 17: 6754.952380952381
Score poblacion 18: 6746.07142857143
Score poblacion 19: 6748.4126984127
Score poblacion 20: 6765.916666666666
Score poblacion 21: 6749.246031746031
Score poblacion 22: 6748.9702380952385
Score poblacion 23: 6758.126984126985
Score poblacion 24: 6745.928571428569
Score poblacion 25: 6791.690476190477
Score poblacion 26: 6740.821428571429
Score poblacion 27: